In [157]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## 3- Se tiene un RDD con información de vuelos programados con la forma (número de vuelo,                código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de               salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada             HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron                 ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de              salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al              estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá                 información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información                de la llegada. Se pide resolver utilizando PySpark: A. Cuál es el aeropuerto con mayor tránsito. B. Cuál es la aerolínea con mayor cantidad de vuelos. C. Cuál es la aerolínea con mayor cantidad de cancelaciones. D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida. E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada. F. Cuál es la aerolínea más puntual.

In [0]:
vuelos_estados_doc  =[(98,'AS','1/3/2020','13:45','3/4/2020','1:30','delayed'),
(2336,'AA','1/3/2020','14:30','1/3/2020','16:30','normal'),
(840,'US','1/3/2020','15:30','1/3/2020','18:30','normal'),
(258,'AA','2/4/2020','16:35','2/4/2020','21:58','delayed'),
(135,'AS','3/4/2020','17:30','3/4/2020','21:45','normal'),
(806,'DL','3/4/2020','18:55','4/4/2020','6:50','delayed'),
(612,'NK','1/3/2020','1:50','1/3/2020','7:20','delayed'),
(2013,'US','5/3/2020','2:20','6/3/2020','1:20','normal'),
(1112,'AA','1/3/2020','3:20','2/4/2020','1:20','normal'),
(1173,'DL','2/4/2020','4:20','3/4/2020','13:30','normal'),
(2336,'DL','','','','','cancelled'),
(1674,'AA','5/3/2020','8:20','','','on course'),
(1434,'DL','','','','','cancelled'),
(2324,'DL','1/3/2020','12:00','1/3/2020','16:00','delayed'),
(2440,'DL','1/3/2020','13:00','2/4/2020','16:00','delayed'),
(108,'AS','3/4/2020','14:00','4/4/2020','5:20','delayed'),
(1560,'DL','5/3/2020','15:00','','','on course'),
(1197,'UA','2/4/2020','16:00','3/4/2020','3:20','normal'),
(122,'AS','5/3/2020','17:00','6/3/2020','3:20','normal'),
(1670,'DL','','','','','cancelled'),
(520,'NK','5/3/2020','21:45','6/4/2020','6:20','normal'),
(371,'AA','2/4/2020','22:45','3/4/2020','','on course'),
(214,'NK','5/3/2020','23:45','6/3/2020','1:45','normal'),
(115,'AA','6/3/2020','0:45','6/3/2020','7:15','normal'),
(1450,'DL','','','','','cancelled'),
(1545,'UA','3/4/2020','6:15','3/4/2020','9:15','normal'),
(130,'AS','5/3/2020','7:15','6/3/2020','17:00','delayed'),
(597,'NK','6/3/2020','8:15','','','on course'),
(413,'US','3/4/2020','9:15','5/3/2020','6:20','normal')]

vuelos_programados_doc = [(98,'AS','ANC','SEA','1/3/2020','13:30','2/4/2020','1:20'),
(2336,'AA','LAX','PBI','1/3/2020','14:30','1/3/2020','16:30'),
(840,'US','SFO','CLT','1/3/2020','15:30','1/3/2020','18:30'),
(258,'AA','LAX','MIA','2/4/2020','16:30','2/4/2020','21:45'),
(135,'AS','SEA','ANC','3/4/2020','17:30','3/4/2020','21:45'),
(806,'DL','SFO','MSP','3/4/2020','18:30','4/4/2020','6:20'),
(612,'NK','LAS','MSP','1/3/2020','1:20','1/3/2020','6:20'),
(2013,'US','LAX','CLT','5/3/2020','2:20','6/3/2020','1:20'),
(1112,'AA','SFO','DFW','1/3/2020','3:20','2/4/2020','1:20'),
(1173,'DL','LAS','ATL','2/4/2020','4:20','3/4/2020','13:30'),
(2336,'DL','DEN','ATL','3/4/2020','5:20','4/4/2020','13:30'),
(1674,'AA','LAS','MIA','5/3/2020','6:20','6/3/2020','11:00'),
(1434,'DL','LAX','MSP','2/4/2020','11:00','3/4/2020','4:20'),
(2324,'DL','SLC','ATL','1/3/2020','12:00','1/3/2020','15:00'),
(2440,'DL','SEA','MSP','1/3/2020','13:00','2/4/2020','15:00'),
(108,'AS','ANC','SEA','3/4/2020','14:00','4/4/2020','4:20'),
(1560,'DL','ANC','SEA','5/3/2020','15:00','6/3/2020','4:20'),
(1197,'UA','SFO','IAH','2/4/2020','16:00','3/4/2020','3:20'),
(122,'AS','ANC','PDX','5/3/2020','17:00','6/3/2020','3:20'),
(1670,'DL','PDX','MSP','3/4/2020','18:00','4/4/2020','23:45'),
(520,'NK','LAS','MCI','5/3/2020','21:45','6/4/2020','6:20'),
(371,'AA','SEA','MIA','2/4/2020','22:45','3/4/2020','0:45'),
(214,'NK','LAS','DFW','5/3/2020','23:45','6/3/2020','1:45'),
(115,'AA','LAX','MIA','6/3/2020','0:45','6/3/2020','7:15'),
(1450,'DL','LAS','MSP','2/4/2020','1:45','3/4/2020','21:45'),
(1545,'UA','LAX','IAH','3/4/2020','6:15','3/4/2020','9:15'),
(130,'AS','FAI','SEA','5/3/2020','7:15','6/3/2020','16:00'),
(597,'NK','MSP','FLL','6/3/2020','8:15','6/3/2020','16:00'),
(413,'US','LAS','CLT','3/4/2020','9:15','4/4/2020','6:20')]

In [0]:
vuelos_programadosRDD = sc.parallelize(vuelos_programados_doc)
vuelos_estadosRDD = sc.parallelize(vuelos_estados_doc)

In [185]:
# (codigo_vuelo, codigo_aerolinea, codigo_aerop_salida, cod_aerop_llegada, fecha_salida, hora_salida, fecha_llegada, hora_llegada)
vuelos_programadosRDD.collect()

[(98, 'AS', 'ANC', 'SEA', '1/3/2020', '13:30', '2/4/2020', '1:20'),
 (2336, 'AA', 'LAX', 'PBI', '1/3/2020', '14:30', '1/3/2020', '16:30'),
 (840, 'US', 'SFO', 'CLT', '1/3/2020', '15:30', '1/3/2020', '18:30'),
 (258, 'AA', 'LAX', 'MIA', '2/4/2020', '16:30', '2/4/2020', '21:45'),
 (135, 'AS', 'SEA', 'ANC', '3/4/2020', '17:30', '3/4/2020', '21:45'),
 (806, 'DL', 'SFO', 'MSP', '3/4/2020', '18:30', '4/4/2020', '6:20'),
 (612, 'NK', 'LAS', 'MSP', '1/3/2020', '1:20', '1/3/2020', '6:20'),
 (2013, 'US', 'LAX', 'CLT', '5/3/2020', '2:20', '6/3/2020', '1:20'),
 (1112, 'AA', 'SFO', 'DFW', '1/3/2020', '3:20', '2/4/2020', '1:20'),
 (1173, 'DL', 'LAS', 'ATL', '2/4/2020', '4:20', '3/4/2020', '13:30'),
 (2336, 'DL', 'DEN', 'ATL', '3/4/2020', '5:20', '4/4/2020', '13:30'),
 (1674, 'AA', 'LAS', 'MIA', '5/3/2020', '6:20', '6/3/2020', '11:00'),
 (1434, 'DL', 'LAX', 'MSP', '2/4/2020', '11:00', '3/4/2020', '4:20'),
 (2324, 'DL', 'SLC', 'ATL', '1/3/2020', '12:00', '1/3/2020', '15:00'),
 (2440, 'DL', 'SEA', 'MSP

In [186]:
# (codigo_vuelo, codigo_aerolinea, fecha_salida, hora_salida, fecha_llegada, hora_llegada, status)
vuelos_estadosRDD.collect()

[(98, 'AS', '1/3/2020', '13:45', '3/4/2020', '1:30', 'delayed'),
 (2336, 'AA', '1/3/2020', '14:30', '1/3/2020', '16:30', 'normal'),
 (840, 'US', '1/3/2020', '15:30', '1/3/2020', '18:30', 'normal'),
 (258, 'AA', '2/4/2020', '16:35', '2/4/2020', '21:58', 'delayed'),
 (135, 'AS', '3/4/2020', '17:30', '3/4/2020', '21:45', 'normal'),
 (806, 'DL', '3/4/2020', '18:55', '4/4/2020', '6:50', 'delayed'),
 (612, 'NK', '1/3/2020', '1:50', '1/3/2020', '7:20', 'delayed'),
 (2013, 'US', '5/3/2020', '2:20', '6/3/2020', '1:20', 'normal'),
 (1112, 'AA', '1/3/2020', '3:20', '2/4/2020', '1:20', 'normal'),
 (1173, 'DL', '2/4/2020', '4:20', '3/4/2020', '13:30', 'normal'),
 (2336, 'DL', '', '', '', '', 'cancelled'),
 (1674, 'AA', '5/3/2020', '8:20', '', '', 'on course'),
 (1434, 'DL', '', '', '', '', 'cancelled'),
 (2324, 'DL', '1/3/2020', '12:00', '1/3/2020', '16:00', 'delayed'),
 (2440, 'DL', '1/3/2020', '13:00', '2/4/2020', '16:00', 'delayed'),
 (108, 'AS', '3/4/2020', '14:00', '4/4/2020', '5:20', 'delayed

In [164]:
# A. Cuál es el aeropuerto con mayor tránsito
# Genero dos RDDs con los codigos de aeropuertos de partida y otro de llegada
aeropuertos_salidas = (vuelos_programadosRDD.map(lambda x: (x[2],1))).reduceByKey(lambda x,y: x + y)
aeropuertos_salidas.collect()

[('LAX', 6),
 ('SFO', 4),
 ('SEA', 3),
 ('LAS', 7),
 ('DEN', 1),
 ('PDX', 1),
 ('FAI', 1),
 ('MSP', 1),
 ('ANC', 4),
 ('SLC', 1)]

In [165]:
aeropuertos_llegadas = (vuelos_programadosRDD.map(lambda x: (x[3],1))).reduceByKey(lambda x,y: x + y)
aeropuertos_llegadas.collect()

[('SEA', 4),
 ('PBI', 1),
 ('CLT', 3),
 ('MSP', 6),
 ('ATL', 3),
 ('IAH', 2),
 ('PDX', 1),
 ('FLL', 1),
 ('MIA', 4),
 ('ANC', 1),
 ('DFW', 2),
 ('MCI', 1)]

In [166]:
# Juntamos los RDD para contabilizar los movimientos totales (salidas y llegadas)
aeropuertos_transito = (aeropuertos_salidas.union(aeropuertos_llegadas)).reduceByKey(lambda x,y: x + y).sortBy(lambda x: -x[1])
aeropuertos_transito.collect()

[('SEA', 7),
 ('LAS', 7),
 ('MSP', 7),
 ('LAX', 6),
 ('ANC', 5),
 ('SFO', 4),
 ('MIA', 4),
 ('CLT', 3),
 ('ATL', 3),
 ('PDX', 2),
 ('IAH', 2),
 ('DFW', 2),
 ('DEN', 1),
 ('FAI', 1),
 ('PBI', 1),
 ('FLL', 1),
 ('SLC', 1),
 ('MCI', 1)]

In [167]:
# Aeropuertos con mayor transito
aeropuertos_transito.take(3)

[('SEA', 7), ('LAS', 7), ('MSP', 7)]

In [168]:
# B. Cuál es la aerolínea con mayor cantidad de vuelos?
# Genero un RDD con las aerolineas y contabilizamos el total de vuelos (planificados)
aerolineas_cantidad_vuelos = (vuelos_programadosRDD.map(lambda x: (x[1],1))).reduceByKey(lambda x,y: x + y).sortBy(lambda x: -x[1])
aerolineas_cantidad_vuelos.take(1)

[('DL', 9)]

In [169]:
# C. Cuál es la aerolínea con mayor cantidad de cancelaciones?
aerolineas_cantidad_cancelaciones = (vuelos_estadosRDD.map(lambda x: (x[1],(1 if x[6]=='cancelled' else 0 )))).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1])
aerolineas_cantidad_cancelaciones.take(1)

[('DL', 4)]

In [170]:
#  D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida?
# Generamos los RDDs con los datos necesarios (no consideramos los vuelos cancelados)
vuelos_retraso_salida = (vuelos_estadosRDD.map(lambda x: (x[0],x[3]))).filter(lambda x: x[1]!='')
vuelos_retraso_salida.collect()

[(98, '13:45'),
 (2336, '14:30'),
 (840, '15:30'),
 (258, '16:35'),
 (135, '17:30'),
 (806, '18:55'),
 (612, '1:50'),
 (2013, '2:20'),
 (1112, '3:20'),
 (1173, '4:20'),
 (1674, '8:20'),
 (2324, '12:00'),
 (2440, '13:00'),
 (108, '14:00'),
 (1560, '15:00'),
 (1197, '16:00'),
 (122, '17:00'),
 (520, '21:45'),
 (371, '22:45'),
 (214, '23:45'),
 (115, '0:45'),
 (1545, '6:15'),
 (130, '7:15'),
 (597, '8:15'),
 (413, '9:15')]

In [171]:
# Generamos los valores horarios en dos campos distintos para poder operarlos
vuelos_retraso_salida_formato = vuelos_retraso_salida.map(lambda x: (x[0], x[1].split(':')))
vuelos_retraso_salida_formato.collect()

[(98, ['13', '45']),
 (2336, ['14', '30']),
 (840, ['15', '30']),
 (258, ['16', '35']),
 (135, ['17', '30']),
 (806, ['18', '55']),
 (612, ['1', '50']),
 (2013, ['2', '20']),
 (1112, ['3', '20']),
 (1173, ['4', '20']),
 (1674, ['8', '20']),
 (2324, ['12', '00']),
 (2440, ['13', '00']),
 (108, ['14', '00']),
 (1560, ['15', '00']),
 (1197, ['16', '00']),
 (122, ['17', '00']),
 (520, ['21', '45']),
 (371, ['22', '45']),
 (214, ['23', '45']),
 (115, ['0', '45']),
 (1545, ['6', '15']),
 (130, ['7', '15']),
 (597, ['8', '15']),
 (413, ['9', '15'])]

In [172]:
# El otro RDD
vuelos_plan_salida = vuelos_programadosRDD.map(lambda x: (x[0],x[5]))
vuelos_plan_salida.collect()

[(98, '13:30'),
 (2336, '14:30'),
 (840, '15:30'),
 (258, '16:30'),
 (135, '17:30'),
 (806, '18:30'),
 (612, '1:20'),
 (2013, '2:20'),
 (1112, '3:20'),
 (1173, '4:20'),
 (2336, '5:20'),
 (1674, '6:20'),
 (1434, '11:00'),
 (2324, '12:00'),
 (2440, '13:00'),
 (108, '14:00'),
 (1560, '15:00'),
 (1197, '16:00'),
 (122, '17:00'),
 (1670, '18:00'),
 (520, '21:45'),
 (371, '22:45'),
 (214, '23:45'),
 (115, '0:45'),
 (1450, '1:45'),
 (1545, '6:15'),
 (130, '7:15'),
 (597, '8:15'),
 (413, '9:15')]

In [173]:
# Generamos los valores horarios en dos campos distintos para poder operarlos
vuelos_plan_salida_formato = vuelos_plan_salida.map(lambda x: (x[0], x[1].split(':')))
vuelos_plan_salida_formato.collect()

[(98, ['13', '30']),
 (2336, ['14', '30']),
 (840, ['15', '30']),
 (258, ['16', '30']),
 (135, ['17', '30']),
 (806, ['18', '30']),
 (612, ['1', '20']),
 (2013, ['2', '20']),
 (1112, ['3', '20']),
 (1173, ['4', '20']),
 (2336, ['5', '20']),
 (1674, ['6', '20']),
 (1434, ['11', '00']),
 (2324, ['12', '00']),
 (2440, ['13', '00']),
 (108, ['14', '00']),
 (1560, ['15', '00']),
 (1197, ['16', '00']),
 (122, ['17', '00']),
 (1670, ['18', '00']),
 (520, ['21', '45']),
 (371, ['22', '45']),
 (214, ['23', '45']),
 (115, ['0', '45']),
 (1450, ['1', '45']),
 (1545, ['6', '15']),
 (130, ['7', '15']),
 (597, ['8', '15']),
 (413, ['9', '15'])]

In [195]:
# Se obtiene un nuevo RDD donde el primer elemento corresponde a el tiempo real y el segundo campo al planificado
diferencia_real_vs_plan_salida =  vuelos_retraso_salida_formato.join(vuelos_plan_salida_formato)
diferencia_real_vs_plan_salida.collect()

[(2336, (['14', '30'], ['14', '30'])),
 (2336, (['14', '30'], ['5', '20'])),
 (840, (['15', '30'], ['15', '30'])),
 (612, (['1', '50'], ['1', '20'])),
 (1112, (['3', '20'], ['3', '20'])),
 (2324, (['12', '00'], ['12', '00'])),
 (2440, (['13', '00'], ['13', '00'])),
 (108, (['14', '00'], ['14', '00'])),
 (1560, (['15', '00'], ['15', '00'])),
 (520, (['21', '45'], ['21', '45'])),
 (2013, (['2', '20'], ['2', '20'])),
 (1173, (['4', '20'], ['4', '20'])),
 (1197, (['16', '00'], ['16', '00'])),
 (1545, (['6', '15'], ['6', '15'])),
 (597, (['8', '15'], ['8', '15'])),
 (413, (['9', '15'], ['9', '15'])),
 (98, (['13', '45'], ['13', '30'])),
 (258, (['16', '35'], ['16', '30'])),
 (806, (['18', '55'], ['18', '30'])),
 (1674, (['8', '20'], ['6', '20'])),
 (122, (['17', '00'], ['17', '00'])),
 (214, (['23', '45'], ['23', '45'])),
 (130, (['7', '15'], ['7', '15'])),
 (135, (['17', '30'], ['17', '30'])),
 (371, (['22', '45'], ['22', '45'])),
 (115, (['0', '45'], ['0', '45']))]

In [0]:
# La comparacion sr hace campo a campo (horaReal vs horaPlan y minReal vs minPlan)
# Se calculan las diferencias horarias de la siguiente forma: (horaReal - horaPlanificada)*60 + (minReal - minPlanificado)
# El resultado de la mora se da en minutos
vuelos_moras_en_minutos_salida = (diferencia_real_vs_plan_salida.map(lambda x: (x[0], ((int(x[1][0][0]) - int(x[1][1][0]))*60) + (int(x[1][0][1]) - int(x[1][1][1])))))

In [197]:
fechas_vuelos = vuelos_programadosRDD.map(lambda x: (x[0],x[4]))
vuelos_moras_por_fecha_en_minutos_salida_aux = vuelos_moras_en_minutos_salida.join(fechas_vuelos)
vuelos_moras_por_fecha_en_minutos_salida = (vuelos_moras_por_fecha_en_minutos_salida_aux.map(lambda x: (str(x[0]) + '-' + x[1][1], x[1][0]))).sortBy(lambda x: -x[1])
vuelos_moras_por_fecha_en_minutos_salida.take(1)

[('2336-1/3/2020', 550)]

In [198]:
# E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada?
vuelos_retraso_llegada = (vuelos_estadosRDD.map(lambda x: (x[0],x[5]))).filter(lambda x: x[1]!='')
vuelos_retraso_llegada.collect()

[(98, '1:30'),
 (2336, '16:30'),
 (840, '18:30'),
 (258, '21:58'),
 (135, '21:45'),
 (806, '6:50'),
 (612, '7:20'),
 (2013, '1:20'),
 (1112, '1:20'),
 (1173, '13:30'),
 (2324, '16:00'),
 (2440, '16:00'),
 (108, '5:20'),
 (1197, '3:20'),
 (122, '3:20'),
 (520, '6:20'),
 (214, '1:45'),
 (115, '7:15'),
 (1545, '9:15'),
 (130, '17:00'),
 (413, '6:20')]

In [199]:
# Generamos los valores horarios en dos campos distintos para poder operarlos
vuelos_retraso_llegada_formato = vuelos_retraso_llegada.map(lambda x: (x[0], x[1].split(':')))
vuelos_retraso_llegada_formato.collect()

[(98, ['1', '30']),
 (2336, ['16', '30']),
 (840, ['18', '30']),
 (258, ['21', '58']),
 (135, ['21', '45']),
 (806, ['6', '50']),
 (612, ['7', '20']),
 (2013, ['1', '20']),
 (1112, ['1', '20']),
 (1173, ['13', '30']),
 (2324, ['16', '00']),
 (2440, ['16', '00']),
 (108, ['5', '20']),
 (1197, ['3', '20']),
 (122, ['3', '20']),
 (520, ['6', '20']),
 (214, ['1', '45']),
 (115, ['7', '15']),
 (1545, ['9', '15']),
 (130, ['17', '00']),
 (413, ['6', '20'])]

In [200]:
# El otro RDD
vuelos_plan_llegada = vuelos_programadosRDD.map(lambda x: (x[0],x[7]))
vuelos_plan_llegada.collect()

[(98, '1:20'),
 (2336, '16:30'),
 (840, '18:30'),
 (258, '21:45'),
 (135, '21:45'),
 (806, '6:20'),
 (612, '6:20'),
 (2013, '1:20'),
 (1112, '1:20'),
 (1173, '13:30'),
 (2336, '13:30'),
 (1674, '11:00'),
 (1434, '4:20'),
 (2324, '15:00'),
 (2440, '15:00'),
 (108, '4:20'),
 (1560, '4:20'),
 (1197, '3:20'),
 (122, '3:20'),
 (1670, '23:45'),
 (520, '6:20'),
 (371, '0:45'),
 (214, '1:45'),
 (115, '7:15'),
 (1450, '21:45'),
 (1545, '9:15'),
 (130, '16:00'),
 (597, '16:00'),
 (413, '6:20')]

In [201]:
# Generamos los valores horarios en dos campos distintos para poder operarlos
vuelos_plan_llegada_formato = vuelos_plan_llegada.map(lambda x: (x[0], x[1].split(':')))
vuelos_plan_llegada_formato.collect()

[(98, ['1', '20']),
 (2336, ['16', '30']),
 (840, ['18', '30']),
 (258, ['21', '45']),
 (135, ['21', '45']),
 (806, ['6', '20']),
 (612, ['6', '20']),
 (2013, ['1', '20']),
 (1112, ['1', '20']),
 (1173, ['13', '30']),
 (2336, ['13', '30']),
 (1674, ['11', '00']),
 (1434, ['4', '20']),
 (2324, ['15', '00']),
 (2440, ['15', '00']),
 (108, ['4', '20']),
 (1560, ['4', '20']),
 (1197, ['3', '20']),
 (122, ['3', '20']),
 (1670, ['23', '45']),
 (520, ['6', '20']),
 (371, ['0', '45']),
 (214, ['1', '45']),
 (115, ['7', '15']),
 (1450, ['21', '45']),
 (1545, ['9', '15']),
 (130, ['16', '00']),
 (597, ['16', '00']),
 (413, ['6', '20'])]

In [202]:
# Se obtiene un nuevo RDD donde el primer elemento corresponde a el tiempo real y el segundo campo al planificado de llegada
diferencia_real_vs_plan_llegada =  vuelos_retraso_llegada_formato.join(vuelos_plan_llegada_formato)
diferencia_real_vs_plan_llegada.collect()

[(2336, (['16', '30'], ['16', '30'])),
 (2336, (['16', '30'], ['13', '30'])),
 (840, (['18', '30'], ['18', '30'])),
 (612, (['7', '20'], ['6', '20'])),
 (1112, (['1', '20'], ['1', '20'])),
 (2324, (['16', '00'], ['15', '00'])),
 (2440, (['16', '00'], ['15', '00'])),
 (108, (['5', '20'], ['4', '20'])),
 (520, (['6', '20'], ['6', '20'])),
 (2013, (['1', '20'], ['1', '20'])),
 (1173, (['13', '30'], ['13', '30'])),
 (1197, (['3', '20'], ['3', '20'])),
 (1545, (['9', '15'], ['9', '15'])),
 (413, (['6', '20'], ['6', '20'])),
 (98, (['1', '30'], ['1', '20'])),
 (258, (['21', '58'], ['21', '45'])),
 (806, (['6', '50'], ['6', '20'])),
 (122, (['3', '20'], ['3', '20'])),
 (214, (['1', '45'], ['1', '45'])),
 (130, (['17', '00'], ['16', '00'])),
 (135, (['21', '45'], ['21', '45'])),
 (115, (['7', '15'], ['7', '15']))]

In [203]:
# La comparacion sr hace campo a campo (horaReal vs horaPlan y minReal vs minPlan)
# Se calculan las diferencias horarias de llegada de la siguiente forma: (horaReal - horaPlanificada)*60 + (minReal - minPlanificado)
# El resultado de la mora se da en minutos
vuelos_moras_en_minutos_llegada = (diferencia_real_vs_plan.map(lambda x: (x[0], ((int(x[1][0][0]) - int(x[1][1][0]))*60) + (int(x[1][0][1]) - int(x[1][1][1])))))
vuelos_moras_en_minutos_llegada.collect()

[(2336, 0),
 (2336, 550),
 (840, 0),
 (612, 30),
 (1112, 0),
 (2324, 0),
 (2440, 0),
 (108, 0),
 (1560, 0),
 (520, 0),
 (2013, 0),
 (1173, 0),
 (1197, 0),
 (1545, 0),
 (597, 0),
 (413, 0),
 (98, 15),
 (258, 5),
 (806, 25),
 (1674, 120),
 (122, 0),
 (214, 0),
 (130, 0),
 (135, 0),
 (371, 0),
 (115, 0)]

In [204]:
vuelos_moras_por_fecha_en_minutos_llegada_aux = vuelos_moras_en_minutos_llegada.join(fechas_vuelos)
vuelos_moras_llegada_por_fecha_en_minutos = (vuelos_moras_por_fecha_en_minutos_llegada_aux.map(lambda x: (str(x[0]) + '-' + x[1][1], x[1][0]))).sortBy(lambda x: -x[1])
vuelos_moras_llegada_por_fecha_en_minutos.take(1)

[('2336-1/3/2020', 550)]

In [231]:
vuelos_moras_totales = (vuelos_moras_en_minutos_llegada.join(vuelos_moras_en_minutos_salida))
vuelos_moras_totales.collect()

[(2336, (0, 0)),
 (2336, (0, 550)),
 (2336, (550, 0)),
 (2336, (550, 550)),
 (840, (0, 0)),
 (1112, (0, 0)),
 (2440, (0, 0)),
 (1560, (0, 0)),
 (520, (0, 0)),
 (1545, (0, 0)),
 (98, (15, 15)),
 (258, (5, 5)),
 (1674, (120, 120)),
 (122, (0, 0)),
 (130, (0, 0)),
 (371, (0, 0)),
 (115, (0, 0)),
 (612, (30, 30)),
 (2324, (0, 0)),
 (108, (0, 0)),
 (2013, (0, 0)),
 (1173, (0, 0)),
 (1197, (0, 0)),
 (597, (0, 0)),
 (413, (0, 0)),
 (806, (25, 25)),
 (214, (0, 0)),
 (135, (0, 0))]

In [243]:
# F. Cuál es la aerolínea más puntual?
# 'Mas puntual' implica que salio y llego segun lo programado
# Genero el RDD que suma las moras calculadas antes por vuelo (moras de llegada y salida)
sumatoria_moras_por_vuelo = vuelos_moras_totales.map(lambda x: (x[0] , (x[1][0] + x[1][1])))
sumatoria_moras_por_vuelo.collect()

[(2336, 0),
 (2336, 550),
 (2336, 550),
 (2336, 1100),
 (840, 0),
 (1112, 0),
 (2440, 0),
 (1560, 0),
 (520, 0),
 (1545, 0),
 (98, 30),
 (258, 10),
 (1674, 240),
 (122, 0),
 (130, 0),
 (371, 0),
 (115, 0),
 (612, 60),
 (2324, 0),
 (108, 0),
 (2013, 0),
 (1173, 0),
 (1197, 0),
 (597, 0),
 (413, 0),
 (806, 50),
 (214, 0),
 (135, 0)]

In [246]:
# Genero un RDD que asocie los vuelos con las aerolineas
vuelo_por_aerolinea = vuelos_estadosRDD.map(lambda x: (x[0],x[1]))
vuelo_por_aerolinea.collect()

[(98, 'AS'),
 (2336, 'AA'),
 (840, 'US'),
 (258, 'AA'),
 (135, 'AS'),
 (806, 'DL'),
 (612, 'NK'),
 (2013, 'US'),
 (1112, 'AA'),
 (1173, 'DL'),
 (2336, 'DL'),
 (1674, 'AA'),
 (1434, 'DL'),
 (2324, 'DL'),
 (2440, 'DL'),
 (108, 'AS'),
 (1560, 'DL'),
 (1197, 'UA'),
 (122, 'AS'),
 (1670, 'DL'),
 (520, 'NK'),
 (371, 'AA'),
 (214, 'NK'),
 (115, 'AA'),
 (1450, 'DL'),
 (1545, 'UA'),
 (130, 'AS'),
 (597, 'NK'),
 (413, 'US')]

In [252]:
# Se asocian vuelos y moras ordenadas de la menor mora a la mayor (de las aerolineas mas puntuales a las menos impuntuales)
moras_por_aerolinea_aux = sumatoria_moras_por_vuelo.join(vuelo_por_aerolinea)
moras_por_aerolinea = (moras_por_aerolinea_aux.map(lambda x: (x[1][1],x[1][0]))).sortBy(lambda x: x[1])
moras_por_aerolinea.collect()

[('US', 0),
 ('DL', 0),
 ('DL', 0),
 ('NK', 0),
 ('AS', 0),
 ('AA', 0),
 ('AA', 0),
 ('AS', 0),
 ('US', 0),
 ('DL', 0),
 ('US', 0),
 ('DL', 0),
 ('NK', 0),
 ('UA', 0),
 ('AA', 0),
 ('AS', 0),
 ('AA', 0),
 ('DL', 0),
 ('UA', 0),
 ('NK', 0),
 ('AS', 0),
 ('AA', 10),
 ('AS', 30),
 ('DL', 50),
 ('NK', 60),
 ('AA', 240),
 ('AA', 550),
 ('DL', 550),
 ('AA', 550),
 ('DL', 550),
 ('AA', 1100),
 ('DL', 1100)]

# FIN DEL EJERCICIO